# Set-Up

In [ ]:
# Install requirements

#!pip install -r requirements.txt

In [3]:
# Import packages

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Personal Data Scraper (Manual Login)

In [ ]:
def scrape_vinted_favourites(max_pages=3, save_csv=True):
    options = Options()
    # 🧠 Don't use headless so you can manually log in
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=options)

    try:
        # 1. Go to Vinted login page
        driver.get("https://www.vinted.co.uk/member/signup/select_type?ref_url=%2F")
        print("🔐 Please manually log in to your Vinted account...")

        # 2. Wait for manual login
        input("✅ Press Enter here once you are logged in...")

        # 3. Go to Favourites page
        results = []
        for page in range(1, max_pages + 1):
            print(f"📄 Scraping favourites page {page}...")
            driver.get(f"https://www.vinted.co.uk/favourites?page={page}")
            time.sleep(3)  # wait for JS to load

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            items = soup.find_all("div", class_="feed-grid__item")

            if not items:
                print(f"⚠️ No items found on page {page}")
                break

            for item in items:
                link_tag = item.find("a", href=True)
                img_tag = item.find("img")

                item_url = "https://www.vinted.co.uk" + link_tag["href"] if link_tag else None
                image_url = img_tag["src"] if img_tag else None
                alt_text = img_tag.get("alt", "").strip()

                results.append({
                    "title": alt_text,
                    "item_url": item_url,
                    "image_url": image_url
                })

        df = pd.DataFrame(results)
        print(f"✅ Scraped {len(df)} items.")

        if save_csv:
            df.to_csv("vinted_favourites.csv", index=False)
            print("💾 Saved to vinted_favourites.csv")

        return df

    finally:
        driver.quit()

In [ ]:
df = scrape_vinted_favourites(max_pages=5)

🔐 Please manually log in to your Vinted account...


KeyboardInterrupt: Interrupted by user

In [ ]:
# Display df

display(df)